In [ ]:
pip install keras
pip install --upgrade tensorflow
conda install -c conda-forge librosa
pip install numpy
pip install scipy
conda install scikit-learn

In [2]:
import numpy as np
import librosa

def wav2mfcc(file_path, max_pad_len=11):
    wave, sr=librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    pad_width = max_pad_len - mfcc.shape[1]
    mfcc = pd.pad(mfcc, pad_width=((0,0), (0, pad_width)), mode='constant')
    return mfcc

In [3]:
import os
DATA_PATH = "./data_section_5.3/"
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [4]:
def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels, _, _= get_labels(path)
    
    for label in labels:
        mfcc_vectors = []
        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)

In [5]:
from sklearn.model_selection import train_test_split

def get_train_test(split_ratio=0.6, random_state=42):
    labels, indices, _ = get_labels(DATA_PATH)
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
    assert X.shape[0] == len(y)
    return train_test_split(X, y, test_size= (1-split_ratio), random_state=random_state, shuffle=True)